In [1]:
import h5netcdf
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import hvplot.xarray
import cartopy.crs as ccrs
import cartopy.feature as cfeat
import earthaccess
from earthaccess import Auth, DataCollections, DataGranules, Store

/Users/katiekohlman/opt/anaconda3/lib/python3.9/site-packages/pyproj/__init__.py:95: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [2]:
auth = earthaccess.login(strategy="interactive", persist=True)


## MUR

In [3]:
mur_results = earthaccess.search_data(short_name = 'MUR-JPL-L4-GLOB-v4.1', temporal = ('2024-11-11', '2024-11-26'), bounding_box = ('-145','-0.5','-135','5'))
ds_mur = xr.open_mfdataset(earthaccess.open(mur_results), engine = 'h5netcdf')
# we want the sea surface temperature variable for this visualization
ds = ds_mur['analysed_sst']

# Subset the dataset so that the program can run the code better
ds_subset = ds.sel(time=slice('2024-11-12T09:00:00', '2024-11-25T09:00:00'))
lat_range = slice(-0.5, 5)
lon_range = slice(-145, -135)
ds_subset = ds_subset.sel(lat=lat_range, lon=lon_range)

QUEUEING TASKS | :   0%|          | 0/17 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/17 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/17 [00:00<?, ?it/s]

In [5]:
ds_subset.to_netcdf("/Volumes/external/MOTIVE/data/MUR/MUR_sst_subset_20241112_20241124.nc")


## MODIS - needs work

In [6]:
import earthaccess
import xarray as xr

# === Search for MODIS Aqua SST granules ===
modis_results = earthaccess.search_data(
    short_name="MODIS_A-JPL-L2P-v2019.0",   # Aqua MODIS
    temporal=("2024-11-11", "2024-11-26"),
    bounding_box=('-145','-0.5','-135','5')
)

QUEUEING TASKS | :   0%|          | 0/57 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/57 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/57 [00:00<?, ?it/s]

ValueError: cannot reindex or align along dimension 'nj' because of conflicting dimension sizes: {2040, 2030}

In [8]:
import xarray as xr
import numpy as np

# === Define region of interest ===
lat_min, lat_max = -0.5, 5
lon_min, lon_max = -145, -135


modis_files = earthaccess.open(modis_results)
roi_datasets = []

for f in modis_files:
    ds = xr.open_dataset(f, engine="h5netcdf")
    sst = ds["sea_surface_temperature"]
    lat = ds["lat"]
    lon = ds["lon"]

    # Mask to ROI
    roi_mask = (
        (lat >= lat_min) & (lat <= lat_max) &
        (lon >= lon_min) & (lon <= lon_max)
    )
    sst_roi = sst.where(roi_mask)

    # Save subset (keeping native swath grid)
    roi_datasets.append(sst_roi)


QUEUEING TASKS | :   0%|          | 0/57 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/57 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/57 [00:00<?, ?it/s]

In [15]:
import xesmf as xe
import os


regridded_ssts = []

for sst_roi in roi_datasets:
    # make sure SST is 2D (y, x)
    sst_vals = np.squeeze(sst_roi.values)
    lat_vals = np.squeeze(sst_roi.lat.values)
    lon_vals = np.squeeze(sst_roi.lon.values)

    ds_in = xr.Dataset(
        {
            "SST": (["y", "x"], sst_vals),
            "lat": (["y", "x"], lat_vals),
            "lon": (["y", "x"], lon_vals),
        }
    )

    # Create regridder (bilinear or nearest neighbor)
    weights_file = "modis_to_target_bilinear.nc"
    regridder = xe.Regridder(
        ds_in, grid_out, "bilinear",
        filename=weights_file,
        reuse_weights=os.path.exists(weights_file)
    )


    # Apply regridding
    sst_reg = regridder(ds_in["SST"])
    regridded_ssts.append(sst_reg)

# Stack all regridded swaths along 'time'
sst_combined = xr.concat(regridded_ssts, dim="time")

# Attach time from original swaths
times = [np.squeeze(sst.time.values) for sst in roi_datasets]
sst_combined = sst_combined.assign_coords(time=("time", times))

# Save
sst_combined.to_netcdf(
    "/Volumes/external/MOTIVE/data/MODIS/MODIS_sst_subset_20241112_20241124.nc"
)


In [16]:
sst_combined

<xarray.DataArray (time: 57, lat: 550, lon: 1000)> Size: 125MB
array([[[      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan, 297.13278, 297.15576, ...,       nan,       nan,
         291.1259 ],
        [      nan, 297.257  , 297.28674, ..., 293.07208, 292.7201 ,
               nan],
        ...,
        [      nan, 260.0041 , 259.99377, ..., 297.31403, 297.0357 ,
         296.44232],
        [      nan, 260.11115, 259.88257, ..., 297.09833, 296.41   ,
         295.82343],
        [      nan,       nan, 261.65952, ...,       nan, 294.46072,
         294.1538 ]],

       [[299.7517 , 299.70874, 299.63922, ...,       nan,       nan,
               nan],
        [299.69925, 299.71088, 299.74686, ...,       nan,       nan,
               nan],
        [299.6488 , 299.55692, 299.689  , ...,       nan,       nan,
               nan],
...
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan]],

       [[      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        ...,
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan]]], dtype=float32)
Coordinates:
  * lon      (lon) float64 8kB -145.0 -145.0 -145.0 ... -135.0 -135.0 -135.0
  * lat      (lat) float64 4kB -0.5 -0.49 -0.48 -0.47 ... 4.96 4.97 4.98 4.99
  * time     (time) datetime64[ns] 456B 2024-11-11T11:45:01 ... 2024-11-26T22...
Attributes:
    regrid_method:  bilinear